In [ ]:
import openai

class OpenAIWrapper:
    def __init__(self, api_keys):
        self.api_keys = api_keys
        self.clients = [openai.ApiClient(api_key) for api_key in api_keys]

    def set_api_keys(self, api_keys):
        self.api_keys = api_keys
        self.clients = [openai.ApiClient(api_key) for api_key in api_keys]

    def create_completion(self, prompt, chat_mode=False, chat_options=None, model="gpt-3.5-turbo", **kwargs):
        client = self._get_next_client()
        if chat_mode:
            if chat_options is None:
                chat_options = {}
            messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
            messages.extend(chat_options.get("messages", []))
            payload = {
                "model": model,
                "messages": messages,
                "temperature": chat_options.get("temperature", 0.7),
                "max_tokens": chat_options.get("max_tokens", 100),
                "top_p": chat_options.get("top_p", 1.0),
                "n": chat_options.get("n", 1),
                "stop": chat_options.get("stop"),
                "presence_penalty": chat_options.get("presence_penalty", 0.0),
                "frequency_penalty": chat_options.get("frequency_penalty", 0.0)
            }
            payload.update(kwargs)
            return client.create_chat_completion(**payload)
        else:
            return client.create_completion(engine=model, prompt=prompt, **kwargs)

    def create_classification(self, model, examples):
        client = self._get_next_client()
        return client.create_classification(model=model, examples=examples)

    def create_answer(self, question, documents, **kwargs):
        client = self._get_next_client()
        return client.create_answer(question=question, documents=documents, **kwargs)

    def _get_next_client(self):
        if not self.clients:
            raise ValueError("No API keys available.")
        client = self.clients.pop(0)
        self.clients.append(client)
        return client


In [ ]:

# Example usage
api_keys = ["API_KEY_1", "API_KEY_2", "API_KEY_3"]

wrapper = OpenAIWrapper(api_keys)

# Example usage of create_completion for text completion with GPT-3.5
completion_response1 = wrapper.create_completion("Hello, world!", model="gpt-3.5-turbo")

# Example usage of create_completion for text completion with GPT-4
completion_response2 = wrapper.create_completion("Hello, world!", model="gpt-4")

print("GPT-3.5 Completion response:")
print(completion_response1)

print("GPT-4 Completion response:")
print(completion_response2)


In [ ]:
## Load Balancing
api_keys = ["API_KEY_1", "API_KEY_2", "API_KEY_3"]
num_api_keys = len(api_keys)

wrapper = OpenAIWrapper(api_keys)

questions = [
    "What is the capital of France?",
    "Who is the author of 'To Kill a Mockingbird'?",
    "What is the square root of 144?",
    "What is the chemical symbol for gold?",
    "Who painted the Mona Lisa?"
]

results = []
question_index = 0

for i in range(len(questions)):
    api_key = api_keys[i % num_api_keys]  # Select the next API key in a round-robin fashion
    wrapper.set_api_keys([api_key])  # Set the API key for the current iteration

    question = questions[question_index]
    completion_response = wrapper.create_completion(question)
    results.append(completion_response)

    question_index = (question_index + 1) % len(questions)  # Move to the next question

# Process the results as needed
for i, result in enumerate(results):
    print(f"Question: {questions[i]}")
    print("Response:", result)
    print("--------------------")
